In [140]:
import os
import pandas as pd
import numpy as np

In [141]:
base_src = "./drive/MyDrive/machine_learning_data"
friend_src = base_src+"/friend.csv"
df = pd.read_csv(friend_src)
df.head()

,name,age,job
0,John,20,student
1,Jenny,30,developer
2,Nate,30,teacher
3,Julia,40,dentist
4,Brian,45,manager


In [14]:
#group by
abalone_src = base_src + '/abalone.data'

#abalone데이터는 friend데이터 처럼 csv가 아니고 날것의 데이터임.
#header가 미리 지정되어 있지 않았기 때문에(즉, 칼럼 이름이 없기 때문에) None으로 지정후
#names를 통해 컬럼명을 지정하며(만약 칼럼 이름이 0번째 행에 있으면 header=0)
#데이터가 ,로 구분되어 있기 때문에 sep(',')를 통해 데이터 각각을 구분함
abalone_df = pd.read_csv(abalone_src,sep=',',
                         header=None,
                         names=['sex','length','diameter','height', 
                  'whole_weight','shucked_weight','viscera_weight', 
                  'shell_weight','rings'])
abalone_df.head()

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [19]:
# 데이터 분석을 통해 shape를 먼저 확인
print(abalone_df.shape)
print('-'*50)

#데이터 결측값 확인
#모든 컬럼에 대한 데이터를 확인하기는 어려우므로
#sum을 통해 얼마나 결측값이 있는지 확인할 수 있음(axis=0, axis=1을 통해 행 기준으로도 확인 가능)
print(abalone_df.isnull().sum())
print('-'*50)

#데이터의 기술통계 확인(numeric 변수만 표현 가능)
print(abalone_df.describe())

(4177, 9)
--------------------------------------------------
sex               0
length            0
diameter          0
height            0
whole_weight      0
shucked_weight    0
viscera_weight    0
shell_weight      0
rings             0
dtype: int64
--------------------------------------------------
            length     diameter  ...  shell_weight        rings
count  4177.000000  4177.000000  ...   4177.000000  4177.000000
mean      0.523992     0.407881  ...      0.238831     9.933684
std       0.120093     0.099240  ...      0.139203     3.224169
min       0.075000     0.055000  ...      0.001500     1.000000
25%       0.450000     0.350000  ...      0.130000     8.000000
50%       0.545000     0.425000  ...      0.234000     9.000000
75%       0.615000     0.480000  ...      0.329000    11.000000
max       0.815000     0.650000  ...      1.005000    29.000000

[8 rows x 8 columns]


In [143]:
#abalone 'sex'에 따라 group by 함수를 통해 'whole_weight'집계
#DataFrame[집계변수].groupby(DataFrame[집계대상])
#이때, 집계대상변수에 DataFrame을 씌워줘야하는데,
#이는 앞의 집계변수에 따른 객체가 해당 집계대상 변수를 가지고 있지 않기 때문임
#반환 값은 객체임 
grouped = abalone_df['whole_weight'].groupby(abalone_df['sex']) # == abalone_df.groupby('sex')['whole_weight'].mean()
grouped.mean() #count,sum, mean,std 등등 가능

sex
F    1.046532
I    0.431363
M    0.991459
Name: whole_weight, dtype: float64

In [159]:
#다수의 연속형 변수에 대한 집계
#groupby를 통해 묶은 객체에 메서드 사용
#이 때, 앞에 데이터프레임에 집계대상에 대한 변수가 포함되어 있으므로
#간단하게 집계대상만 입력해도 됨
print(abalone_df.groupby('sex').mean())
print('-'*90)

#새로운 조건에 맞는 변수 추가
abalone_df['length_bool'] = np.where(abalone_df['length']
                                     >abalone_df['length'].median(),
                                     #True일 경우, False일 경우
                                     'length_long','length_short')
print(abalone_df)
print('-'*90)

#그룹 변수를 2개 이상 선택해서 총계처리
#반환값이 묶여진 abalone객체 이므로, 원하는 컬럼 선택도 가능
print(abalone_df.groupby(['sex','length_bool'])['length'].mean())

       length  diameter    height  ...  shell_weight      rings    answer
sex                                ...                                   
F    0.579093  0.454732  0.158011  ...      0.302010  11.129304  0.887529
I    0.427746  0.326494  0.107996  ...      0.128182   7.890462  0.346498
M    0.561391  0.439287  0.151381  ...      0.281969  10.705497  0.842932

[3 rows x 9 columns]
------------------------------------------------------------------------------------------
     sex  length  diameter  height  ...  shell_weight  rings   length_bool  answer
0      M   0.455     0.365   0.095  ...        0.1500     15  length_short   False
1      M   0.350     0.265   0.090  ...        0.0700      7  length_short   False
2      F   0.530     0.420   0.135  ...        0.2100      9  length_short    True
3      M   0.440     0.365   0.125  ...        0.1550     10  length_short    True
4      I   0.330     0.255   0.080  ...        0.0550      7  length_short   False
...   ..     ...   

In [149]:
#중복 데이터 삭제
#중복된 row 확인
print(abalone_df.duplicated().sum())
print('-'*50)

#중복 예제 생성을 위해 가상으로 중복 데이터 생성
new_abalone = abalone_df.iloc[0:2]

#데이터 프레임끼리 합쳐줌
#axis=0: 가로로 붙이겠다는 뜻
new_abalone_df = pd.concat([abalone_df,new_abalone],axis=0)
print(new_abalone_df)
print('-'*50)

#중복 데이터(row)삭제 (column삭제는 불가)
#원래는 맨 마지막 두개가 중복 데이터 True가 되어 맨 마지막 두개가 사라짐
#만약 맨 마지막 두개를 살리고 싶으면 인자로 (keep='last') 
new_abalone_df = new_abalone_df.drop_duplicates(keep='last')
print(new_abalone_df)

0
--------------------------------------------------
     sex  length  diameter  height  ...  shell_weight  rings   length_bool  answer
0      M   0.455     0.365   0.095  ...         0.150     15  length_short   False
1      M   0.350     0.265   0.090  ...         0.070      7  length_short   False
2      F   0.530     0.420   0.135  ...         0.210      9  length_short    True
3      M   0.440     0.365   0.125  ...         0.155     10  length_short    True
4      I   0.330     0.255   0.080  ...         0.055      7  length_short   False
...   ..     ...       ...     ...  ...           ...    ...           ...     ...
4174   M   0.600     0.475   0.205  ...         0.308      9   length_long    True
4175   F   0.625     0.485   0.150  ...         0.296     10   length_long    True
4176   M   0.710     0.555   0.195  ...         0.495     12   length_long    True
0      M   0.455     0.365   0.095  ...         0.150     15  length_short   False
1      M   0.350     0.265   0.090

In [157]:
#NaN(결측치)를 찾아서 다른 값으로 대체
nan_abalone_df = abalone_df.copy()
#loc을 통해 원하는 곳에 데이터 삽입
#np.nan은 NaN을 의미
nan_abalone_df.loc[2,'length'] = np.nan
nan_abalone_df.loc[2,'sex'] = np.nan
print(nan_abalone_df.isnull().sum())
print('-'*50)

#nan값을 원하는 인자값으로 대체함(0을 넣으면 0으로)
#nan_abalone_df.mean()은 컬럼을 인덱스로 갖는 연속형 변수에 대한 시리즈 값이지만,
#괄호 안에 넣으면 자동으로 nan위치에 맞는 컬럼이 찾아서 들어감
#단 아래의 경우에는 연속형이므로 연속형 변수에 대해서만 mean에 대한 값이 적용됨
print(nan_abalone_df.mean())
print('-'*50)

mean_abalone_df = nan_abalone_df.fillna(nan_abalone_df.mean())
print(mean_abalone_df.head())
print('-'*50)

#카테고리형 컬럼의 결측치를 채우는 방법은 카테고리형 변수를 삽입해줘야함
#해당 컬럼의 각 값에 대해 개수를 반환함
print(nan_abalone_df['sex'].value_counts())
print('-'*50)

replaced_abalone_df = nan_abalone_df['sex'].fillna('replaced')
print(replaced_abalone_df)

sex               1
length            1
diameter          0
height            0
whole_weight      0
shucked_weight    0
viscera_weight    0
shell_weight      0
rings             0
length_bool       0
answer            0
dtype: int64
--------------------------------------------------
length            0.523991
diameter          0.407881
height            0.139516
whole_weight      0.828742
shucked_weight    0.359367
viscera_weight    0.180594
shell_weight      0.238831
rings             9.933684
answer            0.697390
dtype: float64
--------------------------------------------------
   sex    length  diameter  height  ...  shell_weight  rings   length_bool  answer
0    M  0.455000     0.365   0.095  ...         0.150     15  length_short   False
1    M  0.350000     0.265   0.090  ...         0.070      7  length_short   False
2  NaN  0.523991     0.420   0.135  ...         0.210      9  length_short    True
3    M  0.440000     0.365   0.125  ...         0.155     10  length_short 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


In [160]:
#apply 함수 활용
#원하는 행과 열에 연산 혹은 function을 적용할수 있음
#행은 사라지고 열 기준 집계: axis=0
#열은 사라지고 행 기준 집계: axis=1

print(abalone_df[['diameter','whole_weight']].head())
print('-'*50)

print(abalone_df[['diameter','whole_weight']].apply(np.sum,axis=0))
print('-'*50)

print(abalone_df[['diameter','whole_weight']].apply(np.average,axis=1))


   diameter  whole_weight
0     0.365        0.5140
1     0.265        0.2255
2     0.420        0.6770
3     0.365        0.5160
4     0.255        0.2050
--------------------------------------------------
diameter        1703.720
whole_weight    3461.656
dtype: float64
--------------------------------------------------
0       0.43950
1       0.24525
2       0.54850
3       0.44050
4       0.23000
         ...   
4172    0.66850
4173    0.70300
4174    0.82550
4175    0.78975
4176    1.25175
Length: 4177, dtype: float64


In [123]:
import math
#세 변수 합
def my_sum(x,y,z):
    return math.ceil((x+y+z))

print(abalone_df[['diameter','height','whole_weight']].head())
print('-'*50)

abalone_df[['diameter','height','whole_weight']].apply(
    #axis=1이므로 열이 사라지고 행 기준 집계가 됨
    #따라서 x에는 한 줄의 row가 입력되므로 3개의 데이터가 리스트 형태로 들어있음
    #만약 axis=0이면 행이 사라지고 열 기준 집계가 되므로
    #x에는 한 줄의 column, 즉 수천개의 데이터가 리스트 형태로 저장
    lambda x:my_sum(x[0],x[1],x[2]),axis=1)

   diameter  height  whole_weight
0     0.365   0.095        0.5140
1     0.265   0.090        0.2255
2     0.420   0.135        0.6770
3     0.365   0.125        0.5160
4     0.255   0.080        0.2050
--------------------------------------------------


0       1
1       1
2       2
3       2
4       1
       ..
4172    2
4173    2
4174    2
4175    2
4176    3
Length: 4177, dtype: int64

In [126]:
def f(x):
    sum_ = x[0] + x[1] + x[2]
    if sum_>1:
        return True
    else:
        return False 

answer = abalone_df[['diameter','height','whole_weight']].apply(
    lambda x: f(x),axis=1)
abalone_df['answer'] = answer
print(abalone_df)

     sex  length  diameter  height  ...  shell_weight  rings   length_bool  answer
0      M   0.455     0.365   0.095  ...        0.1500     15  length_short   False
1      M   0.350     0.265   0.090  ...        0.0700      7  length_short   False
2      F   0.530     0.420   0.135  ...        0.2100      9  length_short    True
3      M   0.440     0.365   0.125  ...        0.1550     10  length_short    True
4      I   0.330     0.255   0.080  ...        0.0550      7  length_short   False
...   ..     ...       ...     ...  ...           ...    ...           ...     ...
4172   F   0.565     0.450   0.165  ...        0.2490     11   length_long    True
4173   M   0.590     0.440   0.135  ...        0.2605     10   length_long    True
4174   M   0.600     0.475   0.205  ...        0.3080      9   length_long    True
4175   F   0.625     0.485   0.150  ...        0.2960     10   length_long    True
4176   M   0.710     0.555   0.195  ...        0.4950     12   length_long    True

[41

In [139]:
#컬럼 내 유니크 값 뽑아서 개수 확인
#카테고리 변수의 경우 value_counts()
print(abalone_df['sex'].value_counts(ascending=True, dropna=True))
print('-'*50)

#두 개의 DataFrame합치기
one_abalone_df = abalone_df.iloc[1:3]
print(one_abalone_df)
print('-'*50)
pd.concat([abalone_df,one_abalone_df],axis=0)

F    1307
I    1342
M    1528
Name: sex, dtype: int64
--------------------------------------------------
  sex  length  diameter  height  ...  shell_weight  rings   length_bool  answer
1   M    0.35     0.265   0.090  ...          0.07      7  length_short   False
2   F    0.53     0.420   0.135  ...          0.21      9  length_short    True

[2 rows x 11 columns]
--------------------------------------------------


,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings,length_bool,answer
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,length_short,False
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,length_short,False
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,length_short,True
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,length_short,True
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,length_short,False
...,...,...,...,...,...,...,...,...,...,...,...
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.308,9,length_long,True
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.296,10,length_long,True
4176,M,0.710,0.555,0.195,1.9485,0.9455,0.3765,0.495,12,length_long,True
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,length_short,False
